## Milestone 2: Descriptive Stats

In [4]:
# Import packages
import pandas as pd
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [6]:
# Import Yelp dataset

file_path = 'csv file path'
df = pd.read_csv(file_path)

In [9]:
# Descriptive Stats

# Data timespan
print('Earliest & Latest Date:')
print('start_date: ', df.date.min())
print('end_date: ', df.date.max())


Earliest & Latest Date:
start_date:  2005-04-11 00:42:37
end_date:  2022-01-19 19:31:01


In [16]:
# 

def descriptive_stats(table_name, table_col):
    print("==============================")
    print(f"Table Name: {table_name}")
    print(f"Table Col: {table_col}")
    print("==============================")
    
    print("")
    
    # Count
    cnt = pysqldf(f"""SELECT "COUNT" as Stat, count({table_col}) as Value 
                FROM {table_name};""")
    
    # Mean
    mean = pysqldf(f"""SELECT "MEAN" as Stat, 
                CAST(AVG({table_col}) as int) as Value 
                FROM {table_name}""")
    
    # Median
    median = pysqldf(f"""SELECT "MEDIAN" as Stat, {table_col} as Value 
                FROM {table_name}
                ORDER BY {table_col}
                LIMIT 1
                OFFSET (SELECT COUNT(*) FROM {table_name}) / 2 """)
    
    # Mode
    mode = pysqldf(f"""SELECT "MODE" as Stat, 
                {table_col} as Value,
                COUNT(*) as Count
                FROM {table_name}
                GROUP BY {table_col}
                ORDER BY COUNT(*) DESC
                LIMIT 1
                """)
    
    # Min and Max
    
    min = pysqldf(f"""SELECT 'MIN' as Stat, MIN({table_col}) as Value FROM {table_name}""")
    max = pysqldf(f"""SELECT 'MAX' as Stat, MAX({table_col}) as Value FROM {table_name}""")
    
    display(pd.concat([cnt, mean, median, min, max]).set_index("Stat"))
    display(mode.set_index('Stat'))
    print("")

In [22]:
# Check the quality of the data for continous data columns
descriptive_stats('df', 'df.stars')
descriptive_stats('df', 'df.review_count')
descriptive_stats('df', 'df.review_stars')

Table Name: df
Table Col: df.stars



,Value
Stat,
COUNT,351634.0
MEAN,3.0
MEDIAN,4.0
MIN,1.0
MAX,5.0


,Value,Count
Stat,,
MODE,4.5,125866



Table Name: df
Table Col: df.review_count



,Value
Stat,
COUNT,351634
MEAN,453
MEDIAN,185
MIN,5
MAX,5721


,Value,Count
Stat,,
MODE,5721,5778



Table Name: df
Table Col: df.review_stars



,Value
Stat,
COUNT,351634
MEAN,3
MEDIAN,5
MIN,1
MAX,5


,Value,Count
Stat,,
MODE,5,176118


In [26]:
# List the cities with the most reviews

pysqldf("""SELECT city, COUNT(*) as total_review
        FROM df
        GROUP BY state
        ORDER BY total_review DESC
        """)

,city,total_review
0,Philadelphia,85155
1,Tampa,56429
2,New Orleans,41923
3,Nashville,31388
4,St. Louis,24925
5,Fishers,23054
6,Santa Barbara,20251
7,Sparks,19637
8,Tucson,19514
9,Pennsville,9441


In [31]:
# List the coffee company popular operating hour

pd.set_option('display.max_colwidth', None)
pysqldf("""SELECT hours, COUNT(*) as total_review
        FROM df
        GROUP BY hours
        ORDER BY total_review DESC
        LIMIT 5
        """)

,hours,total_review
0,"{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'Wednesday': '0:0-0:0', 'Thursday': '0:0-0:0', 'Friday': '0:0-0:0', 'Saturday': '0:0-0:0', 'Sunday': '0:0-0:0'}",18048
1,"{'Monday': '8:0-18:0', 'Tuesday': '8:0-18:0', 'Wednesday': '8:0-18:0', 'Thursday': '8:0-18:0', 'Friday': '8:0-18:0', 'Saturday': '8:0-18:0', 'Sunday': '8:0-18:0'}",6647
2,"{'Monday': '7:0-14:0', 'Tuesday': '7:0-14:0', 'Wednesday': '7:0-14:0', 'Thursday': '7:0-14:0', 'Friday': '7:0-14:0', 'Saturday': '7:0-14:0', 'Sunday': '7:0-14:0'}",4038
3,"{'Monday': '7:0-15:0', 'Tuesday': '7:0-15:0', 'Wednesday': '7:0-15:0', 'Thursday': '7:0-15:0', 'Friday': '7:0-15:0', 'Saturday': '7:0-15:0', 'Sunday': '7:0-15:0'}",3676
4,"{'Monday': '8:0-14:0', 'Tuesday': '8:0-14:0', 'Wednesday': '8:0-14:0', 'Thursday': '8:0-14:0', 'Friday': '8:0-14:0', 'Saturday': '8:0-14:0', 'Sunday': '8:0-14:0'}",3225


In [37]:
# List the cities with average review rating

pysqldf("""SELECT city, AVG(review_stars) as avg_rating, COUNT(*) as total_review
        FROM df
        GROUP BY city
        ORDER BY total_review DESC, avg_rating
        """)

,city,avg_rating,total_review
0,Philadelphia,4.050741,63006
1,New Orleans,4.075420,36091
2,Nashville,3.923305,24252
3,Tampa,3.910321,23584
4,Tucson,3.857586,18685
...,...,...,...
486,Chester,4.000000,5
487,Cumberland,4.000000,5
488,Eastampton Township,4.200000,5
489,Mehlville,4.400000,5


In [34]:
pd.reset_option('display.max_colwidth')
df.head()


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,user_id,review_stars,text,date
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",6_SpY41LIHZuIaiDs5FMKA,4,This is nice little Chinese bakery in the hear...,2014-05-26 01:09:53
1,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",tCXElwhzekJEH6QJe3xs7Q,4,This is the bakery I usually go to in Chinatow...,2013-10-05 15:19:06
2,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",WqfKtI-aGMmvbA9pPUxNQQ,5,"A delightful find in Chinatown! Very clean, an...",2013-10-25 01:34:57
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",3-1va0IQfK-9tUMzfHWfTA,5,I ordered a graduation cake for my niece and i...,2018-05-20 17:58:57
4,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",EouCKoDfzaVG0klEgdDvCQ,4,HK-STYLE MILK TEA: FOUR STARS\n\nNot quite su...,2013-10-25 02:31:35
